<a href="https://colab.research.google.com/github/nicolaiberk/Apart/blob/master/01_Training_PPRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install sklearn

# code is based on Hauke Licht's CAP Model (https://colab.research.google.com/drive/1n7yHr0-lq-hmsXe2sqxLUhq7_4ejLH9o?usp=sharing#scrollTo=CiaoE0V8XZc6)

     |████████████████████████████████| 2.1MB 18.2MB/s 
     |████████████████████████████████| 901kB 46.4MB/s 
     |████████████████████████████████| 3.3MB 49.1MB/s 


In [2]:
# define model
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os
checkpoint_model = 'distilbert-base-german-cased'
model = DistilBertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [3]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/nlpdl/PPRs_cleaned.csv', engine="python")
df = df.reset_index()
df = df.dropna()
# df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [4]:
# train-test split by date
splitdate = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
train = df[df.date < splitdate]
test = df[df.date >= splitdate]


In [5]:
train_texts = list(train["text"])
train_labels = list(train["label"])

test_texts = list(test["text"])
test_labels = list(test["label"])


# there are probably better ways to do this
ulabels = list(set(train_labels))
train_label_dict = {}

for i in range(len(ulabels)):
  train_label_dict[str(ulabels[i])] = i

train_labels = [train_label_dict[str(l)] for l in train_labels]


# and for test set
ulabels = list(set(test_labels))
test_label_dict = {}

for i in range(len(ulabels)):
  test_label_dict[str(ulabels[i])] = i

test_labels = [test_label_dict[str(l)] for l in test_labels]

In [6]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=1234, stratify=train_labels)

print(f"No. obs    -- test: {len(test_labels)}; train: {len(train_labels)}; validation: {len(val_labels)};")
print(f"No. labels -- test: {len(set(test_labels))}; train: {len(set(train_labels))}; validation: {len(set(val_labels))};")

No. obs    -- test: 7052; train: 16522; validation: 4131;
No. labels -- test: 6; train: 6; validation: 6;


In [7]:
# load the tokenizer 
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-german-cased")

# tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [8]:
import torch
class PPRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PPRDataset(train_encodings, train_labels)
val_dataset = PPRDataset(val_encodings, val_labels)
test_dataset = PPRDataset(test_encodings, test_labels)

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }

In [10]:

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=3,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=64,
    # number of warmup steps for learning rate scheduler
    warmup_steps=500,
    # strength of weight decay
    weight_decay=0.01,     
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=250,
)


In [11]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

In [12]:
trainer.train()

Step,Training Loss
250,1.248200
500,0.190900
750,0.067800
1000,0.065000
1250,0.037400
1500,0.031600
1750,0.032900
2000,0.027200
2250,0.015300
2500,0.017200


TrainOutput(global_step=3099, training_loss=0.14158468403790064, metrics={'train_runtime': 2704.7805, 'train_samples_per_second': 1.146, 'total_flos': 1.0263315977367552e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2521948160, 'init_mem_gpu_alloc_delta': 271062528, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -111566848, 'train_mem_gpu_alloc_delta': 805930496, 'train_mem_cpu_peaked_delta': 123863040, 'train_mem_gpu_peaked_delta': 6562938880})

## Validation performance

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
eval_res = trainer.evaluate()

Trainer is attempting to log a value of "[0.97453311 0.98905908 0.99661476 0.99689922 0.99739244 0.99850075]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.98287671 0.98689956 0.99728997 0.99689922 0.99479844 0.999     ]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.96632997 0.99122807 0.99594046 0.99689922 1.         0.998002  ]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[ 297  684  739  645  765 1001]" of type <class 'numpy.ndarray'> for key "eval/n" as a scalar. This invocation of Tensorboard's writer.add_sc

In [14]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in train_label_dict.items()]
evaluated["class"] = [k for k,v in train_label_dict.items()]
evaluated["f1"] = eval_res["eval_f1"]
evaluated["precision"] = eval_res["eval_precision"]
evaluated["recall"] = eval_res["eval_recall"]
evaluated["n"] = eval_res["eval_n"]

In [15]:
evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
5,5,Linke,0.998501,0.999000,0.998002,1001
4,4,SPD,0.997392,0.994798,1.000000,765
3,3,FDP,0.996899,0.996899,0.996899,645
2,2,Greens,0.996615,0.997290,0.995940,739
1,1,Union,0.989059,0.986900,0.991228,684
0,0,AfD,0.974533,0.982877,0.966330,297


In [16]:
trainer.save_model(os.path.join("drive", "MyDrive", "nlpdl", "01_PPR_model.bin"))

## Test set performance

In [27]:
eval_res = trainer.predict(test_dataset)

KeyboardInterrupt: ignored

In [ ]:
eval_res.metrics

In [ ]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in test_label_dict.items()]
evaluated["class"] = [k for k,v in test_label_dict.items()]
evaluated["f1"] = eval_res.metrics["test_f1"]
evaluated["precision"] = eval_res.metrics["test_precision"]
evaluated["recall"] = eval_res.metrics["test_recall"]
evaluated["n"] = eval_res.metrics["test_n"]

evaluated.sort_values(by="f1", ascending=False)

In [29]:
import numpy as np
preds = eval_res.predictions.argmax(-1)
preds_parties = [""]*len(preds)
for k, v in zip(["AfD", "Union", "Greens", "FDP", "SPD", "Linke"], range(6)):
  preds_parties = np.where(preds == v, k, preds_parties)

labels = eval_res.label_ids
labels_parties = [""]*len(labels)
for k, v in zip(["AfD", "Union", "Greens", "FDP", "SPD", "Linke"], range(6)):
  labels_parties = np.where(labels == v, k, labels_parties)

In [30]:
pd.crosstab(labels_parties, preds_parties, rownames = ["True Label"],colnames = ["Predicted Label"])

Predicted Label,AfD,FDP,Greens,Linke,SPD,Union
True Label,,,,,,
AfD,1997,47,4,108,315,76
FDP,5,953,2,2,123,5
Greens,1,0,819,0,0,0
Linke,1,0,0,1225,0,2
SPD,0,0,0,0,659,2
Union,1,0,1,0,0,704
